In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [5]:
spark =  SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/05 13:30:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/05 13:30:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/05 13:30:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/05 13:30:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
data = [("mohan",1,23,"B.E",23000)]